In [1]:
# 필요한 라이브러리 설정
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.testing import make_tensor
from torch.utils.tensorboard import SummaryWriter

from os.path import join
from tqdm import tqdm
import datetime

from CBDtorch.vaelstm import *
from CBDtorch.custom import Dataset4autoencoder
from CBDtorch.dirs import *


In [2]:
######### 설정 영역 ########
# 실험 관련 세팅
exp_name = 'tor_vaelstm_20220520'  # 실험 이름 혹은 오늘 날짜
modelVersion = 'vaelstm_1st_torch'
nameDataset = 'IWALQQ_AE_1st'
dataType = 'angle' # VAE 나 AE 모델에서는 안중요하지만 추후 모델 predict일 때 편하게 하기 위해서 패킹을 이렇게 해둠

# 데이터 feature 정보, 추후에 자동화가 필요할랑가?
seq_len = 101
num_features = 42
#################################
# 여기는 grid로 돌림! 이제 grid 포함이 default!
#################################
list_embedding_dim = {0: 30,} 
list_learningRate = {0: 0.006}  # opt1
list_batch_size = {0: 128}  # opt2
list_lossFunction = {0: "VAE"}  # opt2

totalFold = 5  # total fold, I did 5-fold cross validation
epochs = 5  # total epoch
log_interval = 10  # frequency for saving log file

# 데이터 위치
absDataDir = r'/restricted/project/movelab/bcha/IMUforKnee/preperation/SAVE_dataSet'
dataSetDir = join(absDataDir,nameDataset)
# 모델 위치
SaveDir = r'/restricted/projectnb/movelab/bcha/IMUforKnee/trainedModel'
logDir = r'/restricted/project/movelab/bcha/IMUforKnee/training/logs'

In [4]:
dir_save_torch = join(SaveDir,modelVersion,nameDataset)
# 수정된 방법으로 모델 불러올 떄
# Model class must be defined somewhere
numFold = 0
model = torch.load(join(dir_save_torch,f'{dataType}_{numFold}_fold.pt'))
model.eval() # 항상 필요함!!! 모델을 불렀을 떄는 항상 넣기

RecurrentVariationalAutoencoder(
  (encoder): VariationalEncoder(
    (rnn1): LSTM(42, 60, batch_first=True, bidirectional=True)
    (rnn2): LSTM(120, 30, batch_first=True, bidirectional=True)
    (mu): Linear(in_features=60, out_features=30, bias=True)
    (sigma): Linear(in_features=60, out_features=30, bias=True)
  )
  (decoder): Decoder(
    (rnn1): LSTM(30, 30, batch_first=True, bidirectional=True)
    (rnn2): LSTM(60, 60, batch_first=True, bidirectional=True)
    (output_layer): Linear(in_features=120, out_features=42, bias=True)
    (timedist): TimeDistributed(
      (module): Linear(in_features=120, out_features=42, bias=True)
    )
  )
)

In [5]:
angle_train = Dataset4autoencoder(dataSetDir, dataType, 'train',numFold)
angle_test  = Dataset4autoencoder(dataSetDir, dataType, 'test', numFold)
train_loader = DataLoader(angle_train, batch_size=128, shuffle=True)
test_loader = DataLoader(angle_test, batch_size=128, shuffle=True)

In [7]:
X = next(iter(train_loader))

In [9]:
X_hat = model(X)

In [12]:
X[0].shape, X_hat[0].shape

(torch.Size([101, 42]), torch.Size([101, 42]))